In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('/content/Price_data.csv')

In [3]:
data.head()

,Date,Equity 1,Equity 2,Equity 3,Equity 4
0,31-Jan-20,197.2525,253.2126,653.2660,653.2678
1,3-Feb-20,197.0630,253.2231,652.3695,652.3759
2,4-Feb-20,196.6896,252.9168,649.9793,649.9858
3,5-Feb-20,197.3429,252.8294,653.5655,653.5588
4,6-Feb-20,197.4554,252.7901,652.3171,652.3172


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527 entries, 0 to 526
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       527 non-null    object 
 1   Equity 1   526 non-null    float64
 2   Equity 2   527 non-null    float64
 3   Equity 3   527 non-null    float64
 4   Equity 4   527 non-null    float64
dtypes: float64(4), object(1)
memory usage: 20.7+ KB


In [5]:
data = data.dropna()

In [6]:
data.describe()

,Equity 1,Equity 2,Equity 3,Equity 4
count,526.000000,526.000000,526.000000,526.000000
mean,199.504363,265.477602,858.294516,858.294256
std,8.148380,3.899170,110.541786,110.541505
min,183.650300,252.042700,649.979300,649.985800
25%,191.209100,264.491275,750.675475,750.674375
50%,198.346750,266.051750,873.308200,873.307850
75%,206.508400,267.940450,952.246650,952.247050
max,214.117800,271.146100,1139.933000,1139.934800


In [7]:
pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 13.7 MB/s eta 0:00:00


In [8]:
data

,Date,Equity 1,Equity 2,Equity 3,Equity 4
0,31-Jan-20,197.2525,253.2126,653.2660,653.2678
1,3-Feb-20,197.0630,253.2231,652.3695,652.3759
2,4-Feb-20,196.6896,252.9168,649.9793,649.9858
3,5-Feb-20,197.3429,252.8294,653.5655,653.5588
4,6-Feb-20,197.4554,252.7901,652.3171,652.3172
...,...,...,...,...,...
522,25-Feb-22,204.9000,267.7623,1030.6482,1030.6490
523,28-Feb-22,206.0440,266.5691,1049.7208,1049.7272
524,1-Mar-22,205.3662,265.1722,1092.8045,1092.8039
525,2-Mar-22,206.0620,261.6414,1124.7237,1124.7264


In [9]:
# Update the imports
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Initialize the app
app = dash.Dash(__name__)

# Give the app a layout
app.layout = html.Div([
    html.H1("Stock Performance Viewer"),
    dcc.Dropdown(
        id="stock-dropdown",
        options=[{"label": stock, "value": stock} for stock in data.columns[1:]],
        value=data.columns[1],
    ),
    dcc.DatePickerRange(
        id="date-picker",
        start_date=data["Date"].min(),
        end_date=data["Date"].max(),
    ),
    dcc.Graph(id="stock-graph"),
    html.Div(id="estimated-return"),
])

@app.callback(
    [Output("stock-graph", "figure"),
     Output("estimated-return", "children")],
    [Input("stock-dropdown", "value"),
     Input("date-picker", "start_date"),
     Input("date-picker", "end_date")]
)
def update_graph(stock, start_date, end_date):
    filtered_data = data[(data["Date"] >= start_date) & (data["Date"] <= end_date)]

    # Check if filtered_data is empty
    if filtered_data.empty:
        return px.line(), "No data available"

    # Create line chart with Plotly
    fig = px.line(filtered_data, x="Date", y=stock)
    fig.update_layout(title=f"{stock} Stock Prices", xaxis_title="Date", yaxis_title="Price")

    # Calculate estimated return using the specified formula
    start_price = filtered_data.iloc[0][stock]
    end_price = filtered_data.iloc[-1][stock]
    n = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days
    estimated_return = (((end_price / start_price) ** (1 / (n / 365))) - 1) * 100
    return fig, f"Estimated Return: {estimated_return:.2f}%"


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>